In [17]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell 0: Mount Google Drive
# This cell must be run *first* to authenticate and connect Google Drive.

print("--- [Cell 0] START: Mount Google Drive ---")

import sys
import os
import logging
import time

# Check for logger
if "logger" not in locals():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)
    logger = logging.getLogger(__name__)

start_time_cell = time.time()

try:
    from google.colab import drive

    DRIVE_MOUNT_POINT = "/content/drive"

    if os.path.isdir(os.path.join(DRIVE_MOUNT_POINT, "MyDrive")):
        logger.info(f"Google Drive already mounted at: {DRIVE_MOUNT_POINT}")
        print(f"Google Drive already mounted at: {DRIVE_MOUNT_POINT}")
    else:
        logger.info(f"Mounting Google Drive at: {DRIVE_MOUNT_POINT}...")
        print(f"Mounting Google Drive at: {DRIVE_MOUNT_POINT}...")
        drive.mount(DRIVE_MOUNT_POINT)
        logger.info("Google Drive mounted successfully.")
        print("Google Drive mounted successfully.")

except ImportError:
    logger.warning("Could not import google.colab. Not running in Colab environment.")
    print("WARNING: Not running in Colab environment. Drive mounting skipped.")
except Exception as e:
    logger.error(f"An error occurred during Google Drive mount: {e}")
    print(f"ERROR: An error occurred during Google Drive mount: {e}")

cell_duration = time.time() - start_time_cell
logger.info(f"Cell 0 finished in {cell_duration:.2f}s.")
print(f"\n--- [Cell 0] FINISHED: Mount Google Drive ({cell_duration:.2f}s) ---")



--- [Cell 0] START: Mount Google Drive ---
2025-11-04 21:50:14,024 - INFO - Google Drive already mounted at: /content/drive
Google Drive already mounted at: /content/drive
2025-11-04 21:50:14,025 - INFO - Cell 0 finished in 0.00s.

--- [Cell 0] FINISHED: Mount Google Drive (0.00s) ---


In [8]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell A: Environment & Reproducibility
# This cell sets up the environment, seeds, paths, and global config.

print("--- [Cell A] START: Environment & Reproducibility ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import scipy
import sklearn
import seaborn as sns
import matplotlib
import json
import time
import joblib # Using joblib for more efficient pickle operations

# --- 1. Setup Robust Logger ---
# This logger will write to both console and a file on Google Drive
try:
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    LOG_FILE_PATH = os.path.join(DRIVE_ROOT, "experiment.log")

    # Ensure directory exists (Cell 0 should have done this)
    os.makedirs(DRIVE_ROOT, exist_ok=True)

    # Configure root logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO) # Set root level to INFO

    # Clear existing handlers to avoid duplicates
    if logger.hasHandlers():
        logger.handlers.clear()

    # Create file handler (writes to G-Drive)
    file_handler = logging.FileHandler(LOG_FILE_PATH)
    file_handler.setLevel(logging.INFO)
    file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)

    # Create console handler (writes to Colab output)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO) # Only show INFO and above in console
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(console_formatter)
    logger.addHandler(console_handler)

    print(f"Logger configured. Log file: {LOG_FILE_PATH}")
    logger.info(f"--- [Cell A] Logger configured. Log file: {LOG_FILE_PATH} ---")

except Exception as e:
    print(f"CRITICAL ERROR: Failed to configure logger: {e}")
    # Still create a basic logger just for this cell
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    logger.error(f"Failed to configure file logger: {e}")

# --- 2. Log System & Library Versions ---
try:
    print("\n--- Logging System & Library Versions ---")
    logger.info("--- Logging System & Library Versions ---")

    logger.info(f"Python Version: {sys.version.split()[0]}")
    logger.info(f"Platform: {sys.platform}")
    logger.info(f"Numpy Version: {np.__version__}")
    logger.info(f"Scipy Version: {scipy.__version__}")
    logger.info(f"Pandas Version: {pd.__version__}")
    logger.info(f"Matplotlib Version: {matplotlib.__version__}")
    logger.info(f"SKLearn Version: {sklearn.__version__}")
    logger.info(f"Seaborn Version: {sns.__version__}")
    logger.info(f"Joblib Version: {joblib.__version__}")

    print("System & Library versions logged.")

except Exception as e:
    logger.error(f"Could not log library versions: {e}")

# --- 3. Define Global Configuration & Paths ---
try:
    start_time = time.time()

    # This is the root for all *artifacts* (plots, CSVs)
    OUTPUT_ROOT = DRIVE_ROOT # Main experiment folder on G-Drive
    print(f"Ensured output directory exists: {OUTPUT_ROOT}")
    logger.info(f"Output artifact root set to: {OUTPUT_ROOT}")

    # Define the global CONFIG dictionary
    CONFIG = {
        # --- Paths ---
        "output_root": OUTPUT_ROOT,
        "config_file": os.path.join(OUTPUT_ROOT, "config.json"),
        "log_file": LOG_FILE_PATH,

        # --- State Files (for passing data between cells) ---
        "state_file_B": os.path.join(OUTPUT_ROOT, "state_B_ground_states.pkl"),
        "state_file_C": os.path.join(OUTPUT_ROOT, "state_C_rdm_data.pkl"),
        "state_file_D": os.path.join(OUTPUT_ROOT, "state_D_structure_profiles.pkl"),
        "state_file_E": os.path.join(OUTPUT_ROOT, "state_E_kq_profiles.pkl"),
        "state_file_F": os.path.join(OUTPUT_ROOT, "state_F_mi_matrices.pkl"),
        "state_file_dF": os.path.join(OUTPUT_ROOT, "state_F_dmi_matrices.pkl"), # [FIX] Separate file for d_MI
        "state_file_G": os.path.join(OUTPUT_ROOT, "state_G_h1_results.pkl"),
        "state_file_H": os.path.join(OUTPUT_ROOT, "state_H_h2_results.pkl"),
        "state_file_I": os.path.join(OUTPUT_ROOT, "state_I_h3_results.pkl"),
        "state_file_J": os.path.join(OUTPUT_ROOT, "state_J_h4_results.pkl"),
        "state_file_K_setup": os.path.join(OUTPUT_ROOT, "state_K_setup_complete.pkl"),
        "state_file_K": os.path.join(OUTPUT_ROOT, "state_K_h5_results.pkl"),
        "state_file_L_manifest": os.path.join(OUTPUT_ROOT, "manifest.json"),

        # --- Model Parameters ---
        "N_exact": [6, 8, 10],
        "N_mps": [32, 48, 64], # Per Spec H5
        "g_values": [0.5, 1.0, 1.5], # Pre-critical, critical, post-critical
        "J_coupling": 1.0,

        # --- Analysis Parameters ---
        "RNG_SEED": 20250315,
        "eps_log": 1e-8,       # For -log(MI + eps)
        "eps_path": 1e-4,      # For d_KQ = sum(eps + |grad|)
        "bootstrap_ci": 200,   # Number of resamples for CI
        "mps_max_bond_dim": 256 # Per Spec H5
    }

    print("\n--- Global CONFIG Dictionary Defined ---")
    print(json.dumps(CONFIG, indent=2))

    # Save the config to G-Drive
    config_save_path = CONFIG["config_file"]
    with open(config_save_path, 'w') as f:
        json.dump(CONFIG, f, indent=2)

    print(f"Configuration saved to: {config_save_path}")
    logger.info(f"Configuration saved to: {config_save_path}")

    # --- [DIAGNOSTIC] ---
    print(f"VERIFY: Keys saved to config: {list(CONFIG.keys())}")
    logger.info(f"VERIFY: Keys saved to config: {list(CONFIG.keys())}")
    # --- [END DIAGNOSTIC] ---

    # --- 4. Seed RNGs ---
    RNG_SEED = CONFIG["RNG_SEED"]
    np.random.seed(RNG_SEED)
    print(f"Global NumPy RNG seed set to: {RNG_SEED}")
    logger.info(f"Global NumPy RNG seed set to: {RNG_SEED}")

    total_time = time.time() - start_time
    print(f"\n--- [Cell A] FINISHED: Environment & Reproducibility ({total_time:.2f}s) ---")
    logger.info(f"--- [Cell A] FINISHED ({total_time:.2f}s) ---")

except Exception as e:
    print(f"ERROR in Cell A: {e}")
    logger.error(f"An error occurred in Cell A: {e}")
    import traceback
    logger.error(traceback.format_exc())



--- [Cell A] START: Environment & Reproducibility ---
Logger configured. Log file: /content/drive/MyDrive/qtx_vrd_experiment/experiment.log
2025-11-04 21:45:35,073 - INFO - --- [Cell A] Logger configured. Log file: /content/drive/MyDrive/qtx_vrd_experiment/experiment.log ---

--- Logging System & Library Versions ---
2025-11-04 21:45:35,075 - INFO - --- Logging System & Library Versions ---
2025-11-04 21:45:35,075 - INFO - Python Version: 3.12.12
2025-11-04 21:45:35,076 - INFO - Platform: linux
2025-11-04 21:45:35,077 - INFO - Numpy Version: 2.0.2
2025-11-04 21:45:35,077 - INFO - Scipy Version: 1.16.3
2025-11-04 21:45:35,078 - INFO - Pandas Version: 2.2.2
2025-11-04 21:45:35,079 - INFO - Matplotlib Version: 3.10.0
2025-11-04 21:45:35,080 - INFO - SKLearn Version: 1.6.1
2025-11-04 21:45:35,080 - INFO - Seaborn Version: 0.13.2
2025-11-04 21:45:35,081 - INFO - Joblib Version: 1.5.2
System & Library versions logged.
Ensured output directory exists: /content/drive/MyDrive/qtx_vrd_experiment

In [9]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell B: Hamiltonian & Ground State (ExactDiag)

print("--- [Cell B] START: Hamiltonian & Ground State (ExactDiag) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    # This block ensures that even if run standalone, the script finds the config
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # Import sparse matrices from scipy
    from scipy.sparse import identity, kron, csc_matrix
    from scipy.sparse.linalg import eigsh

    logger.info("Cell B dependencies (CONFIG, logger) loaded.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    sys.exit(1)

# --- 2. Helper Functions (Hamiltonian) ---

def build_tfim_hamiltonian(N, J, g):
    """
    Builds the 1D Transverse-Field Ising Model (TFIM) Hamiltonian
    H = -J * sum(Z_i * Z_{i+1}) - g * sum(X_i)
    """

    # Define Pauli matrices (sparse)
    sigma_x = csc_matrix(np.array([[0, 1], [1, 0]], dtype=complex))
    sigma_z = csc_matrix(np.array([[1, 0], [0, -1]], dtype=complex))
    identity_2 = csc_matrix(np.eye(2, dtype=complex))

    dim = 2**N
    H = csc_matrix((dim, dim), dtype=complex)

    # Interaction term: -J * sum(Z_i * Z_{i+1})
    for i in range(N - 1):
        # Create operator list: [I, I, ..., Z, Z, ..., I]
        # [FIX] Use identity(2) (size of one site)
        ops = [identity_2] * N
        ops[i] = sigma_z
        ops[i+1] = sigma_z

        # Take Kronecker product
        H_term = ops[0]
        for op in ops[1:]:
            H_term = kron(H_term, op)

        H -= J * H_term

    # Transverse field term: -g * sum(X_i)
    for i in range(N):
        # Create operator list: [I, I, ..., X, ..., I]
        # [FIX] Use identity(2)
        ops = [identity_2] * N
        ops[i] = sigma_x

        # Take Kronecker product
        H_term = ops[0]
        for op in ops[1:]:
            H_term = kron(H_term, op)

        H -= g * H_term

    return H.real.tocsc() # Hamiltonian is real

def find_ground_state(H):
    """
    Finds the ground state (eigenvector with smallest eigenvalue)
    of a sparse Hamiltonian H.
    """
    # k=1: find 1 eigenvector
    # [FIX] 'SA' (Smallest Algebraic) is correct for eigsh (symmetric)
    # 'SR' was incorrect and caused the 'LM SM LA SA BE' error.
    eigenvalues, eigenvectors = eigsh(H, k=1, which='SA')

    E0 = eigenvalues[0]
    psi_0 = eigenvectors[:, 0]

    return E0, psi_0

# --- 3. Main Execution: Calculate Ground States ---
try:
    print(f"Starting Exact Diagonalization for {len(CONFIG['N_exact']) * len(CONFIG['g_values'])} configurations...")

    N_values = CONFIG["N_exact"]
    g_values = CONFIG["g_values"]
    J_coupling = CONFIG["J_coupling"]

    logger.info(f"N values: {N_values}, g values: {g_values}")

    ground_states_exact = {} # Dictionary to store results
    summary_data = [] # For CSV

    # Create list of all (N, g) pairs for tqdm
    config_list = [(N, g) for N in N_values for g in g_values]

    for N, g in tqdm(config_list, desc="Calculating Ground States"):
        run_id = f"N{N}_g{g:.1f}"
        logger.info(f"  Processing {run_id}...")

        try:
            # 1. Build Hamiltonian
            t_start_H = time.time()
            H = build_tfim_hamiltonian(N, J_coupling, g)
            t_H = time.time() - t_start_H
            logger.info(f"    Hamiltonian (N={N}, dim={H.shape}) built in {t_H:.4f}s")

            # 2. Find Ground State
            t_start_E0 = time.time()
            E0, psi_0 = find_ground_state(H)
            t_E0 = time.time() - t_start_E0
            logger.info(f"    Ground State found in {t_E0:.4f}s. E0 = {E0:f}")

            # 3. Store results
            ground_states_exact[run_id] = {
                "run_id": run_id,
                "N": N,
                "g": g,
                "E0": E0,
                "psi_0": psi_0,
                "t_H_s": t_H,
                "t_E0_s": t_E0
            }

            summary_data.append({
                "run_id": run_id, "N": N, "g": g, "E0": E0,
                "t_H(s)": t_H, "t_E0(s)": t_E0
            })

        except Exception as e:
            logger.error(f"    Failed to find ground state for {run_id}: {e}")
            logger.error(traceback.format_exc())
            print(f"ERROR on {run_id}: {e}")

    # --- 4. Save & Report Results ---
    print("\n--- Ground State Calculation Summary ---")
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string())

    # Save summary CSV to G-Drive
    summary_csv_path = os.path.join(OUTPUT_ROOT, "E0_summary_exact.csv")
    summary_df.to_csv(summary_csv_path, index=False)
    print(f"\nSummary table saved to: {summary_csv_path}")

    # Save the full ground state data (with vectors) to a pickle file
    state_path = CONFIG["state_file_B"]
    joblib.dump(ground_states_exact, state_path)

    print(f"Full ground state data (with vectors) saved to: {state_path}")

    print(f"\nStored {len(ground_states_exact)} ground states in memory (variable: ground_states_exact).")
    print(f"Example keys: {list(ground_states_exact.keys())[:3]}")
    logger.info(f"Saved ground state data to {state_path}")
    logger.info(f"Saved E0 summary to {summary_csv_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell B: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell B: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell B finished in {total_time:.2f}s.")
    print(f"\n--- [Cell B] FINISHED: Hamiltonian & Ground State ({total_time:.2f}s) ---")



--- [Cell B] START: Hamiltonian & Ground State (ExactDiag) ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:45:40,432 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:45:40,435 - INFO - Cell B dependencies (CONFIG, logger) loaded.
Starting Exact Diagonalization for 9 configurations...
2025-11-04 21:45:40,437 - INFO - N values: [6, 8, 10], g values: [0.5, 1.0, 1.5]


Calculating Ground States:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:45:40,445 - INFO -   Processing N6_g0.5...
2025-11-04 21:45:40,462 - INFO -     Hamiltonian (N=6, dim=(64, 64)) built in 0.0168s
2025-11-04 21:45:40,465 - INFO -     Ground State found in 0.0019s. E0 = -5.522030
2025-11-04 21:45:40,466 - INFO -   Processing N6_g1.0...
2025-11-04 21:45:40,481 - INFO -     Hamiltonian (N=6, dim=(64, 64)) built in 0.0141s
2025-11-04 21:45:40,483 - INFO -     Ground State found in 0.0019s. E0 = -7.296230
2025-11-04 21:45:40,484 - INFO -   Processing N6_g1.5...
2025-11-04 21:45:40,498 - INFO -     Hamiltonian (N=6, dim=(64, 64)) built in 0.0133s
2025-11-04 21:45:40,500 - INFO -     Ground State found in 0.0018s. E0 = -9.847571
2025-11-04 21:45:40,501 - INFO -   Processing N8_g0.5...
2025-11-04 21:45:40,539 - INFO -     Hamiltonian (N=8, dim=(256, 256)) built in 0.0372s
2025-11-04 21:45:40,542 - INFO -     Ground State found in 0.0027s. E0 = -7.640593
2025-11-04 21:45:40,543 - INFO -   Processing N8_g1.0...
2025-11-04 21:45:40,581 - INFO -     

In [10]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell C: Reduced States & Entropies

print("--- [Cell C] START: Reduced States & Entropies ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # Load ground states from Cell B
    state_path = CONFIG["state_file_B"]
    print(f"Loading ground states from: {state_path}...")

    if not os.path.exists(state_path):
        print(f"ERROR: State file from Cell B not found: {state_path}")
        raise FileNotFoundError(f"State file from Cell B not found: {state_path}")

    ground_states_exact = joblib.load(state_path)

    if not ground_states_exact:
        print("ERROR: Ground states data is empty. Please run Cell B first.")
        raise ValueError("Ground states data is empty. Please run Cell B first.")

    logger.info("Cell C dependencies (CONFIG, ground_states_exact, logger) loaded.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    sys.exit(1)

# --- 2. Helper Functions (RDM & Entropy) ---

def get_reduced_density_matrix(psi_0, i, N):
    """
    Calculates the single-site reduced density matrix rho_i for site i.
    psi_0 must be the full state vector (dim 2**N).
    """

    # Reshape the state vector into a tensor of shape (2, 2, ..., 2)
    psi_tensor = psi_0.reshape([2] * N)

    # Axes to contract (all sites *except* i)
    axes_to_contract = [k for k in range(N) if k != i]

    # --- [FIX] Handle edge cases (i=0, i=N-1) and middle cases correctly ---
    if i == 0:
        # Contract all axes from 1 to N-1
        # tensordot(psi, psi_conj, axes=([1, 2], [1, 2]))
        axes = (list(range(1, N)), list(range(1, N)))
        rho_i = np.tensordot(psi_tensor, psi_tensor.conj(), axes=axes)
    elif i == N - 1:
        # Contract all axes from 0 to N-2
        # tensordot(psi, psi_conj, axes=([0, 1], [0, 1]))
        axes = (list(range(0, N - 1)), list(range(0, N - 1)))
        rho_i = np.tensordot(psi_tensor, psi_tensor.conj(), axes=axes)
    else:
        # Site i is in the middle.
        # We need to contract axes [0...i-1] and [i+1...N-1]
        axes_left = list(range(0, i))
        axes_right = list(range(i + 1, N))
        axes_to_contract = (axes_left + axes_right, axes_left + axes_right)
        rho_i = np.tensordot(psi_tensor, psi_tensor.conj(), axes=axes_to_contract)

    # rho_i should now be a (2, 2) matrix.
    # The previous buggy implementation returned (1, 2, 1, 2) or (2, 1, 2, 1) at edges.
    return rho_i

def compute_von_neumann_entropy(rho):
    """Computes S = -Tr(rho * log2(rho))"""
    # [FIX] np.linalg.eigvalsh requires a square matrix.
    # The new get_reduced_density_matrix() ensures rho is (2, 2).
    eigvals = np.linalg.eigvalsh(rho)

    # Filter out zero (or near-zero) eigenvalues to avoid log(0)
    non_zero_eigvals = eigvals[eigvals > 1e-15]

    # S = -sum(p_i * log(p_i))
    entropy_e = -np.sum(non_zero_eigvals * np.log(non_zero_eigvals))

    # Convert from nats (log_e) to bits (log_2)
    return entropy_e / np.log(2)

# --- 3. Main Execution: Calculate Entropies ---
try:
    print(f"Starting RDM & Entropy calculation for {len(ground_states_exact)} configurations...")

    all_rdm_data = {} # To store all results (RDMs, entropies)

    # Use tqdm for a progress bar
    for run_id, data in tqdm(ground_states_exact.items(), desc="Calculating Entropies"):
        N = data["N"]
        g = data["g"]
        psi_0 = data["psi_0"]

        logger.info(f"  Processing {run_id} (N={N})...")

        rdms_list = []
        entropies_S_bits = []

        for i in range(N):
            # 1. Get RDM rho_i
            rho_i = get_reduced_density_matrix(psi_0, i, N)
            rdms_list.append(rho_i)

            # 2. Compute entropy S(i)
            S_bits = compute_von_neumann_entropy(rho_i)
            entropies_S_bits.append(S_bits)

        # H_norm(i) = S(i) / log2(dim) = S(i) / log2(2) = S(i)
        H_norm_list = entropies_S_bits

        # Store results
        all_rdm_data[run_id] = {
            "N": N, "g": g,
            "rdms": {i: rdms_list[i] for i in range(N)}, # Map site index to RDM
            "S_bits_list": entropies_S_bits,
            "H_norm_list": H_norm_list
        }

        # [FIX] Log the correct list (entropies_S_bits)
        logger.info(f"    S(i)_bits (first 3): {[f'{s:.4f}' for s in entropies_S_bits[:3]]}")

        # Save per-run CSV
        run_output_dir = os.path.join(OUTPUT_ROOT, run_id)
        os.makedirs(run_output_dir, exist_ok=True) # Ensure dir exists

        df = pd.DataFrame({
            "site": range(N),
            "S_bits": entropies_S_bits,
            "H_norm": H_norm_list
        })
        df.to_csv(os.path.join(run_output_dir, "entropies.csv"), index=False)

    # --- 4. Save & Report Results ---
    print("\n--- RDM & Entropy Calculation Summary ---")
    print(f"Successfully processed {len(all_rdm_data)} runs.")

    # Save the full RDM data (with matrices) to a pickle file on G-Drive
    state_path = CONFIG["state_file_C"]
    joblib.dump(all_rdm_data, state_path)

    print(f"\nFull RDM/Entropy data saved to: {state_path}")
    print(f"Stored all RDM data in memory (variable: all_rdm_data).")

    # Show example from last run
    if "run_id" in locals(): # Check if loop ran
        last_run_csv = os.path.join(OUTPUT_ROOT, run_id, "entropies.csv")
        if os.path.exists(last_run_csv):
            print(f"\nExample data for last run ({run_id}): {last_run_csv}")
            print(pd.read_csv(last_run_csv).head().to_string())

    print(f"Example keys: {list(all_rdm_data.keys())[:3]}")
    logger.info(f"Saved RDM/Entropy data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell C: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell C: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell C finished in {total_time:.2f}s.")
    print(f"\n--- [Cell C] FINISHED: Reduced States & Entropies ({total_time:.2f}s) ---")



--- [Cell C] START: Reduced States & Entropies ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:45:48,602 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading ground states from: /content/drive/MyDrive/qtx_vrd_experiment/state_B_ground_states.pkl...
2025-11-04 21:45:48,608 - INFO - Cell C dependencies (CONFIG, ground_states_exact, logger) loaded.
Starting RDM & Entropy calculation for 9 configurations...


Calculating Entropies:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:45:48,617 - INFO -   Processing N6_g0.5 (N=6)...
2025-11-04 21:45:48,618 - INFO -     S(i)_bits (first 3): ['0.8108', '0.9275', '0.9390']
2025-11-04 21:45:48,626 - INFO -   Processing N6_g1.0 (N=6)...
2025-11-04 21:45:48,627 - INFO -     S(i)_bits (first 3): ['0.3750', '0.5561', '0.5876']
2025-11-04 21:45:48,634 - INFO -   Processing N6_g1.5 (N=6)...
2025-11-04 21:45:48,635 - INFO -     S(i)_bits (first 3): ['0.1922', '0.3137', '0.3271']
2025-11-04 21:45:48,641 - INFO -   Processing N8_g0.5 (N=8)...
2025-11-04 21:45:48,642 - INFO -     S(i)_bits (first 3): ['0.8205', '0.9347', '0.9463']
2025-11-04 21:45:48,662 - INFO -   Processing N8_g1.0 (N=8)...
2025-11-04 21:45:48,663 - INFO -     S(i)_bits (first 3): ['0.3798', '0.5641', '0.6014']
2025-11-04 21:45:48,670 - INFO -   Processing N8_g1.5 (N=8)...
2025-11-04 21:45:48,671 - INFO -     S(i)_bits (first 3): ['0.1923', '0.3140', '0.3282']
2025-11-04 21:45:48,677 - INFO -   Processing N10_g0.5 (N=10)...
2025-11-04 21:45:48,679

In [20]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell D: Structural Term C(i)

print("--- [Cell D] START: Structural Term C(i) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
import json
import time
import traceback
from tqdm.auto import tqdm
from scipy.sparse import coo_matrix, kron, identity
from sklearn.preprocessing import minmax_scale # For C_norm

start_time = time.time()
logger = logging.getLogger(__name__)

# --- 1. Define Helper Functions (must be self-contained) ---
def get_operator(op, i, N):
    """Helper to build a full-chain operator op at site i."""
    ops = [identity(2) for _ in range(N)]
    ops[i] = op
    full_op = ops[0]
    for k in range(1, N):
        full_op = kron(full_op, ops[k])
    return full_op.tocoo() # Use COO for efficient arithmetic

def get_connected_correlator(psi_0, op1_matrix, i, op2_matrix, j, N):
    """Calculates <op1(i) op2(j)>_c = <op1(i) op2(j)> - <op1(i)><op2(j)>"""

    # 1. Calculate <op1(i) op2(j)>
    op1_full = get_operator(op1_matrix, i, N)
    op2_full = get_operator(op2_matrix, j, N)

    # <psi| op1 * op2 |psi>
    op1_op2_psi = op1_full.dot(op2_full.dot(psi_0))
    exp_op1_op2 = np.dot(psi_0.conj(), op1_op2_psi).real

    # 2. Calculate <op1(i)>
    op1_psi = op1_full.dot(psi_0)
    exp_op1 = np.dot(psi_0.conj(), op1_psi).real

    # 3. Calculate <op2(j)>
    op2_psi = op2_full.dot(psi_0)
    exp_op2 = np.dot(psi_0.conj(), op2_psi).real

    # 4. Connected part
    connected_corr = exp_op1_op2 - (exp_op1 * exp_op2)
    return connected_corr, exp_op1, exp_op2

def get_exp_value_from_rdm(rdm, op_matrix):
    """Calculate <Op> = Tr(rho * Op) from a reduced density matrix."""
    return np.trace(rdm @ op_matrix).real

# --- 2. Load Config & Dependencies ---
try:
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]

    # --- Load state data from Cell B and C ---
    print(f"Loading ground states (B) from: {CONFIG['state_file_B']}...")
    ground_states_exact = joblib.load(CONFIG["state_file_B"])

    print(f"Loading RDM data (C) from: {CONFIG['state_file_C']}...")
    all_rdm_data = joblib.load(CONFIG["state_file_C"])

    print("Cell D dependencies loaded.")
    logger.info("Cell D dependencies loaded.")

    all_structure_profiles = {}

    # Define Pauli matrices
    sigma_X = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_Z = np.array([[1, 0], [0, -1]], dtype=complex)

# --- 3. Main Execution: Calculate C(i) ---
    print(f"Starting Structure Term C(i) calculation for {len(ground_states_exact)} configurations...")

    pbar = tqdm(ground_states_exact.items(), desc="Calculating C(i)")
    for run_id, data in pbar:

        pbar.set_description(f"Processing {run_id}")
        logger.info(f"  Processing C(i) for: {run_id}")

        N = data["N"]
        psi_0 = data["psi_0"]

        if run_id not in all_rdm_data:
            logger.warning(f"    Skipping {run_id}: No RDM data found (Cell C output).")
            continue

        rdms_list = all_rdm_data[run_id]["rdms"]

        C_raw_ZZ_list = []
        C_raw_XX_list = []
        exp_vals_Z = []
        exp_vals_X = []

        # 1. Get all single-site expectation values <Z_i> and <X_i> from RDMs
        for i in range(N):
            rdm_i = rdms_list[i]
            exp_vals_Z.append(get_exp_value_from_rdm(rdm_i, sigma_Z))
            exp_vals_X.append(get_exp_value_from_rdm(rdm_i, sigma_X))

        # 2. Get connected correlators
        for i in range(N):
            # Find neighbors (default: nearest)
            neighbors_j = [j for j in [i - 1, i + 1] if 0 <= j < N]

            if not neighbors_j:
                C_raw_ZZ_list.append(0.0)
                C_raw_XX_list.append(0.0)
                continue

            C_zz_neighbors = []
            C_xx_neighbors = []

            for j in neighbors_j:
                # Calculate <Z_i Z_j>_c = <Z_i Z_j> - <Z_i><Z_j>
                op1_full = get_operator(sigma_Z, i, N)
                op2_full = get_operator(sigma_Z, j, N)
                op1_op2_psi = op1_full.dot(op2_full.dot(psi_0))
                exp_op1_op2 = np.dot(psi_0.conj(), op1_op2_psi).real
                connected_zz = exp_op1_op2 - (exp_vals_Z[i] * exp_vals_Z[j])
                C_zz_neighbors.append(np.abs(connected_zz))

                # Calculate <X_i X_j>_c = <X_i X_j> - <X_i><X_j>
                op1_full = get_operator(sigma_X, i, N)
                op2_full = get_operator(sigma_X, j, N)
                op1_op2_psi = op1_full.dot(op2_full.dot(psi_0))
                exp_op1_op2 = np.dot(psi_0.conj(), op1_op2_psi).real
                connected_xx = exp_op1_op2 - (exp_vals_X[i] * exp_vals_X[j])
                C_xx_neighbors.append(np.abs(connected_xx))

            # Average over neighbors
            C_raw_ZZ_list.append(np.mean(C_zz_neighbors))
            C_raw_XX_list.append(np.mean(C_xx_neighbors))

        # 3. Combine and Normalize
        # C_raw(i) = mean( |<Z_i Z_j>_c|, |<X_i X_j>_c| ) averaged over j
        C_raw_total = np.mean([C_raw_ZZ_list, C_raw_XX_list], axis=0)

        # Min-max normalize C(i) to [0, 1] per configuration
        C_norm_list = minmax_scale(C_raw_total)

        # [SYSTEMIC FIX] Store as a DataFrame, not a dict.
        sites = np.arange(N)
        structure_df = pd.DataFrame({
            "site": sites,
            "C_raw_ZZ": C_raw_ZZ_list,
            "C_raw_XX": C_raw_XX_list,
            "C_norm": C_norm_list
        })

        # Store in memory
        all_structure_profiles[run_id] = structure_df

        # Save to CSV
        run_path = os.path.join(OUTPUT_ROOT, run_id)
        os.makedirs(run_path, exist_ok=True)
        csv_path = os.path.join(run_path, "structure.csv")
        structure_df.to_csv(csv_path, index=False)
        logger.info(f"    Saved structure data to: {csv_path}")

    # --- 4. Save State ---
    state_path = CONFIG["state_file_D"]
    joblib.dump(all_structure_profiles, state_path)

    print("\n--- Structure Term C(i) Summary ---")
    print(f"Successfully processed {len(all_structure_profiles)} runs.")
    print(f"\nExample data for last run ({run_id}): {csv_path}")
    print(all_structure_profiles[run_id].head().to_string())

    print(f"\nFull structure profile data saved to: {state_path}")
    print(f"Stored {len(all_structure_profiles)} profiles in memory (variable: all_structure_profiles).")
    logger.info(f"Saved structure profile data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell D: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell D: {e}")
    sys.exit(1)

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell D finished in {total_time:.2f}s.")
    print(f"\n--- [Cell D] FINISHED: Structural Term C(i) ({total_time:.2f}s) ---")



--- [Cell D] START: Structural Term C(i) ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:10,454 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading ground states (B) from: /content/drive/MyDrive/qtx_vrd_experiment/state_B_ground_states.pkl...
Loading RDM data (C) from: /content/drive/MyDrive/qtx_vrd_experiment/state_C_rdm_data.pkl...
Cell D dependencies loaded.
2025-11-04 21:54:10,466 - INFO - Cell D dependencies loaded.
Starting Structure Term C(i) calculation for 9 configurations...


Calculating C(i):   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:54:10,474 - INFO -   Processing C(i) for: N6_g0.5
2025-11-04 21:54:10,578 - INFO -     Saved structure data to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g0.5/structure.csv
2025-11-04 21:54:10,579 - INFO -   Processing C(i) for: N6_g1.0
2025-11-04 21:54:10,670 - INFO -     Saved structure data to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.0/structure.csv
2025-11-04 21:54:10,671 - INFO -   Processing C(i) for: N6_g1.5
2025-11-04 21:54:10,757 - INFO -     Saved structure data to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.5/structure.csv
2025-11-04 21:54:10,759 - INFO -   Processing C(i) for: N8_g0.5
2025-11-04 21:54:10,943 - INFO -     Saved structure data to: /content/drive/MyDrive/qtx_vrd_experiment/N8_g0.5/structure.csv
2025-11-04 21:54:10,944 - INFO -   Processing C(i) for: N8_g1.0
2025-11-04 21:54:11,163 - INFO -     Saved structure data to: /content/drive/MyDrive/qtx_vrd_experiment/N8_g1.0/structure.csv
2025-11-04 21:54:11,165 - INFO -   Processing C(i)

In [21]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell E: Canonical KQ = C(1 - H_norm)

print("--- [Cell E] START: Canonical KQ Computation & Plots ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
import json
import time
import traceback
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re # Added import for re

start_time = time.time()
logger = logging.getLogger(__name__)

try:
    # --- 1. Load Config & Dependencies ---
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # --- Load state data from Cell C and D ---
    print(f"Loading RDM data (C) from: {CONFIG['state_file_C']}...")
    all_rdm_data = joblib.load(CONFIG["state_file_C"])

    print(f"Loading Structure profiles (D) from: {CONFIG['state_file_D']}...")
    all_structure_profiles = joblib.load(CONFIG["state_file_D"])

    print("Cell E dependencies loaded.")
    logger.info("Cell E dependencies loaded.")

    all_kq_profiles = {}

    # --- 2. Main Execution: Calculate KQ ---
    print(f"Starting KQ = C(1-H_norm) calculation for {len(all_rdm_data)} configurations...")

    pbar = tqdm(all_rdm_data.items(), desc="Calculating KQ")
    for run_id, rdm_data in pbar:

        pbar.set_description(f"Processing {run_id}")
        logger.info(f"  Processing KQ for: {run_id}")

        if run_id not in all_structure_profiles:
            logger.warning(f"    Skipping {run_id}: No structure data found (Cell D output).")
            continue

        # Get H_norm from Cell C data
        H_norm_list = rdm_data["H_norm_list"]
        H_norm = np.array(H_norm_list)

        # Get C_norm from Cell D data
        C_norm_list = all_structure_profiles[run_id]["C_norm"]
        C_norm = np.array(C_norm_list)

        N = len(H_norm)
        if len(C_norm) != N:
            logger.error(f"    Skipping {run_id}: Mismatch in length H_norm ({N}) and C_norm ({len(C_norm)}).")
            continue

        # --- Canonical KQ Calculation ---
        KQ = C_norm * (1 - H_norm)

        sites = np.arange(N)

        # [SYSTEMIC FIX] Store as a DataFrame, not a dict.
        kq_df = pd.DataFrame({
            "site": sites,
            "H_norm": H_norm,
            "C_norm": C_norm,
            "KQ": KQ
        })

        # Store in memory
        all_kq_profiles[run_id] = kq_df

        # --- Save to Disk ---
        run_path = os.path.join(OUTPUT_ROOT, run_id)
        os.makedirs(run_path, exist_ok=True)

        csv_path = os.path.join(run_path, "kq.csv")
        kq_df.to_csv(csv_path, index=False)
        logger.info(f"    Saved KQ to: {csv_path}")

        # --- Plotting ---
        try:
            N_val = int(re.search(r'N(\d+)', run_id).group(1))
            g_val = float(re.search(r'g([\d\.]+)', run_id).group(1))

            fig, ax = plt.subplots(figsize=(10, 6))
            sns.lineplot(x="site", y="H_norm", data=kq_df, marker='o', label=r"$H_{norm}$ (Entropy)", ax=ax)
            sns.lineplot(x="site", y="C_norm", data=kq_df, marker='s', label=r"$C$ (Structure)", ax=ax)
            sns.lineplot(x="site", y="KQ", data=kq_df, marker='x', label=r"KQ = C(1-$H_{norm}$)", ax=ax, lw=2.5, color='black')

            ax.set_title(rf"Per-Site Profiles (N={N_val}, g={g_val:.1f})")
            ax.set_xlabel("Site (i)")
            ax.set_ylabel("Normalized Value")
            ax.grid(True, linestyle='--', alpha=0.6)
            ax.legend()

            plot_path = os.path.join(run_path, "profiles.png")
            plt.savefig(plot_path, dpi=100, bbox_inches='tight')
            plt.close(fig)
            logger.info(f"    Saved profile plot to: {plot_path}")

        except Exception as plot_e:
            logger.error(f"    Failed to generate plot for {run_id}: {plot_e}")

    # --- 3. Save State ---
    state_path = CONFIG["state_file_E"]
    joblib.dump(all_kq_profiles, state_path)
    print(f"\nFull KQ profile data saved to: {state_path}")
    print(f"Stored {len(all_kq_profiles)} KQ profiles in memory (variable: all_kq_profiles).")
    logger.info(f"Saved KQ profile data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell E: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell E: {e}")
    sys.exit(1) # Exit on any failure

finally:
    # --- 4. Cleanup & Timing (always runs) ---
    total_time = time.time() - start_time
    logger.info(f"Cell E finished in {total_time:.2f}s.")
    print(f"\n--- [Cell E] FINISHED: Canonical KQ Computation & Plots ({total_time:.2f}s) ---")



--- [Cell E] START: Canonical KQ Computation & Plots ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:20,200 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading RDM data (C) from: /content/drive/MyDrive/qtx_vrd_experiment/state_C_rdm_data.pkl...
Loading Structure profiles (D) from: /content/drive/MyDrive/qtx_vrd_experiment/state_D_structure_profiles.pkl...
Cell E dependencies loaded.
2025-11-04 21:54:20,211 - INFO - Cell E dependencies loaded.
Starting KQ = C(1-H_norm) calculation for 9 configurations...


Calculating KQ:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:54:20,219 - INFO -   Processing KQ for: N6_g0.5
2025-11-04 21:54:20,227 - INFO -     Saved KQ to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g0.5/kq.csv
2025-11-04 21:54:20,494 - INFO -     Saved profile plot to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g0.5/profiles.png
2025-11-04 21:54:20,496 - INFO -   Processing KQ for: N6_g1.0
2025-11-04 21:54:20,504 - INFO -     Saved KQ to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.0/kq.csv
2025-11-04 21:54:20,761 - INFO -     Saved profile plot to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.0/profiles.png
2025-11-04 21:54:20,762 - INFO -   Processing KQ for: N6_g1.5
2025-11-04 21:54:20,770 - INFO -     Saved KQ to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.5/kq.csv
2025-11-04 21:54:21,026 - INFO -     Saved profile plot to: /content/drive/MyDrive/qtx_vrd_experiment/N6_g1.5/profiles.png
2025-11-04 21:54:21,028 - INFO -   Processing KQ for: N8_g0.5
2025-11-04 21:54:21,036 - INFO -     Saved KQ to: /content/dri

In [22]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell F: MI Matrix & Distances

print("--- [Cell F] START: MI Matrix & Distances ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]

    # --- [DIAGNOSTIC] ---
    print(f"VERIFY: Keys loaded from config: {list(CONFIG.keys())}")
    logger.info(f"VERIFY: Keys loaded from config: {list(CONFIG.keys())}")
    if "state_file_dF" not in CONFIG:
        print("CRITICAL ERROR: 'state_file_dF' not found in loaded config. Run Cell A again.")
        logger.error("CRITICAL ERROR: 'state_file_dF' not found in loaded config. Run Cell A again.")
        raise KeyError("'state_file_dF' not found in config. Run Cell A again.")
    # --- [END DIAGNOSTIC] ---

    # --- [FIX END] ---

    # Load ground states (B) and RDM data (C)
    state_path_B = CONFIG["state_file_B"]
    state_path_C = CONFIG["state_file_C"]

    print(f"Loading ground states (B) from: {state_path_B}...")
    logger.info(f"Loading ground states from: {state_path_B}...")
    if not os.path.exists(state_path_B):
        raise FileNotFoundError(f"State file from Cell B not found: {state_path_B}")
    ground_states_exact = joblib.load(state_path_B)

    print(f"Loading RDM data (C) from: {state_path_C}...")
    logger.info(f"Loading RDM data from: {state_path_C}...")
    if not os.path.exists(state_path_C):
        raise FileNotFoundError(f"State file from Cell C not found: {state_path_C}")
    all_rdm_data = joblib.load(state_path_C)

    if not ground_states_exact or not all_rdm_data:
        raise ValueError("Loaded data is empty. Please run Cells B and C.")

    logger.info("Successfully loaded dependencies for Cell F.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    logger.error(traceback.format_exc())
    sys.exit(1)

# --- 2. Helper Functions (MI) ---

# [FIX] Removed @np.vectorize
# This function takes a 2D matrix, not a scalar.
def compute_von_neumann_entropy(rho):
    """Computes S = -Tr(rho * log2(rho))"""
    eigvals = np.linalg.eigvalsh(rho)
    non_zero_eigvals = eigvals[eigvals > 1e-15]
    entropy_e = -np.sum(non_zero_eigvals * np.log(non_zero_eigvals))
    return entropy_e / np.log(2)

def get_bipartite_rdm(psi_0, sites_A, N):
    """
    Calculates the RDM for a subsystem defined by sites_A (list of indices).
    """
    sites_B = [i for i in range(N) if i not in sites_A]

    # Reshape psi_0 into a tensor
    psi_tensor = psi_0.reshape([2] * N)

    # Contract axes corresponding to subsystem B
    # tensordot(psi, psi_conj, axes=(sites_B, sites_B))
    rho_A = np.tensordot(psi_tensor, psi_tensor.conj(), axes=(sites_B, sites_B))

    # Reshape into a 2D density matrix
    dim_A = 2**len(sites_A)
    rho_A_matrix = rho_A.reshape(dim_A, dim_A)

    return rho_A_matrix

def calculate_all_mi(N, entropies_S_bits, psi_0):
    """
    Calculates the full MI(i,j) matrix.
    MI(i,j) = S(i) + S(j) - S(i,j)
    """
    mi_matrix = np.zeros((N, N))

    # Get all pairs (i, j) where i < j
    pair_list = list(itertools.combinations(range(N), 2))

    for i, j in tqdm(pair_list, desc=f"  MI pairs (N={N})", leave=False):
        # 1. Get S(i) and S(j) (already computed)
        S_i = entropies_S_bits[i]
        S_j = entropies_S_bits[j]

        # 2. Compute S(i,j)
        rho_ij = get_bipartite_rdm(psi_0, [i, j], N)
        S_ij = compute_von_neumann_entropy(rho_ij)

        # 3. Compute MI
        mi = S_i + S_j - S_ij

        # MI must be non-negative (handle numerical noise)
        mi_matrix[i, j] = max(0.0, mi)
        mi_matrix[j, i] = max(0.0, mi) # Symmetric

    return mi_matrix

# --- 3. Main Execution: Calculate MI(i,j) ---
try:
    print(f"Starting MI Matrix calculation for {len(all_rdm_data)} configurations...")
    logger.info(f"--- [Cell F] START ---")

    all_mi_matrices = {}
    all_dmi_matrices = {} # For d_MI = -log(MI)
    mi_epsilon = CONFIG.get("eps_log", 1e-8)
    logger.info(f"Using mi_epsilon (eps_log) = {mi_epsilon}")

    for run_id, data_C in tqdm(all_rdm_data.items(), desc="Calculating MI(i,j)"):

        # [FIX] Get psi_0 from the correct dictionary
        if run_id not in ground_states_exact:
            logger.warning(f"  Skipping {run_id}: No ground state data found.")
            continue

        N = data_C["N"]
        g = data_C["g"]
        psi_0 = ground_states_exact[run_id]["psi_0"]
        entropies_S_bits = data_C["S_bits_list"]

        logger.info(f"  Processing {run_id} (N={N})... (This may take a while)")

        # Calculate full MI matrix
        mi_matrix = calculate_all_mi(N, entropies_S_bits, psi_0)

        # Calculate d_MI matrix
        d_mi_matrix = -np.log(mi_matrix + mi_epsilon)

        # Store results
        all_mi_matrices[run_id] = mi_matrix
        all_dmi_matrices[run_id] = d_mi_matrix

        # --- Save artifacts ---
        run_output_dir = os.path.join(OUTPUT_ROOT, run_id)

        # Save matrices (CSV)
        mi_path = os.path.join(run_output_dir, "mi_matrix.csv")
        dmi_path = os.path.join(run_output_dir, "dmi_matrix.csv")
        np.savetxt(mi_path, mi_matrix, delimiter=",")
        np.savetxt(dmi_path, d_mi_matrix, delimiter=",")

        # Save heatmap plot
        try:
            plt.figure(figsize=(10, 8))
            sns.heatmap(mi_matrix, annot=True, fmt=".2f", cmap="viridis")
            plt.title(f"Mutual Information MI(i,j) (N={N}, g={g:.1f})")
            plt.xlabel("Site (j)")
            plt.ylabel("Site (i)")

            plot_path = os.path.join(run_output_dir, "mi_heatmap.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            plt.close()

            logger.info(f"    Saved MI heatmap and matrices for {run_id}")
        except Exception as e:
            logger.error(f"    Failed to generate MI heatmap for {run_id}: {e}")

    # --- 4. Save & Report Results ---
    print("\n--- MI Matrix Calculation Summary ---")
    print(f"Successfully processed {len(all_mi_matrices)} runs.")

    # Save the full data to pickle files on G-Drive
    # [FIX] Save MI and d_MI to *separate* files
    state_path_F = CONFIG["state_file_F"]
    state_path_dF = CONFIG["state_file_dF"]

    joblib.dump(all_mi_matrices, state_path_F)
    joblib.dump(all_dmi_matrices, state_path_dF)

    print(f"\nFull MI(i,j) matrix data saved to: {state_path_F}")
    print(f"Full d_MI(i,j) matrix data saved to: {state_path_dF}")
    print(f"Stored matrices in memory (variables: all_mi_matrices, all_dmi_matrices).")

    logger.info(f"Saved MI matrix data to {state_path_F}")
    logger.info(f"Saved d_MI matrix data to {state_path_dF}")

except Exception as e:
    logger.error(f"An error occurred in Cell F: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell F: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell F finished in {total_time:.2f}s.")
    print(f"\n--- [Cell F] FINISHED: MI Matrix & Distances ({total_time:.2f}s) ---")



--- [Cell F] START: MI Matrix & Distances ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:25,167 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
VERIFY: Keys loaded from config: ['output_root', 'config_file', 'log_file', 'state_file_B', 'state_file_C', 'state_file_D', 'state_file_E', 'state_file_F', 'state_file_dF', 'state_file_G', 'state_file_H', 'state_file_I', 'state_file_J', 'state_file_K_setup', 'state_file_K', 'state_file_L_manifest', 'N_exact', 'N_mps', 'g_values', 'J_coupling', 'RNG_SEED', 'eps_log', 'eps_path', 'bootstrap_ci', 'mps_max_bond_dim']
2025-11-04 21:54:25,170 - INFO - VERIFY: Keys loaded from config: ['output_root', 'config_file', 'log_file', 'state_file_B', 'state_file_C', 'state_file_D', 'state_file_E', 'state_file_F', 'state_file_dF', 'state_file_G', 'state_file_H', 'state_file_I', 'state_file_J', 'state_file_K_setup', 'state_file_K', 'state_file_L_manifest', 'N_exact', 

Calculating MI(i,j):   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:54:25,190 - INFO -   Processing N6_g0.5 (N=6)... (This may take a while)


  MI pairs (N=6):   0%|          | 0/15 [00:00<?, ?it/s]

2025-11-04 21:54:25,558 - INFO -     Saved MI heatmap and matrices for N6_g0.5
2025-11-04 21:54:25,560 - INFO -   Processing N6_g1.0 (N=6)... (This may take a while)


...

2025-11-04 21:54:28,009 - INFO -     Saved MI heatmap and matrices for N10_g0.5
2025-11-04 21:54:28,011 - INFO -   Processing N10_g1.0 (N=10)... (This may take a while)


  MI pairs (N=10):   0%|          | 0/45 [00:00<?, ?it/s]

2025-11-04 21:54:28,485 - INFO -     Saved MI heatmap and matrices for N10_g1.0
2025-11-04 21:54:28,487 - INFO -   Processing N10_g1.5 (N=10)... (This may take a while)


  MI pairs (N=10):   0%|          | 0/45 [00:00<?, ?it/s]

2025-11-04 21:54:28,950 - INFO -     Saved MI heatmap and matrices for N10_g1.5

--- MI Matrix Calculation Summary ---
Successfully processed 9 runs.

Full MI(i,j) matrix data saved to: /content/drive/MyDrive/qtx_vrd_experiment/state_F_mi_matrices.pkl
Full d_MI(i,j) matrix data saved to: /content/drive/MyDrive/qtx_vrd_experiment/state_F_dmi_matrices.pkl
Stored matrices in memory (variables: all_mi_matrices, all_dmi_matrices).
2025-11-04 21:54:28,964 - INFO - Saved MI matrix data to /content/drive/MyDrive/qtx_vrd_experiment/state_F_mi_matrices.pkl
2025-11-04 21:54:28,964 - INFO - Saved d_MI matrix data to /content/drive/MyDrive/qtx_vrd_experiment/state_F_dmi_matrices.pkl
2025-11-04 21:54:28,965 - INFO - Cell F finished in 3.80s.

--- [Cell F] FINISHED: MI Matrix & Distances (3.80s) ---


In [23]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell G: Local Geometry Test (H1)

print("--- [Cell G] START: Local Geometry Test (H1) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import minmax_scale
import re # Import regex for parsing run_id

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # Load KQ profiles (E) and d_MI matrices (F)
    state_path_E = CONFIG["state_file_E"]
    state_path_dF = CONFIG["state_file_dF"] # [FIX] Load d_MI, not MI

    print(f"Loading KQ profiles (E) from: {state_path_E}...")
    if not os.path.exists(state_path_E):
        raise FileNotFoundError(f"State file from Cell E not found: {state_path_E}")
    all_kq_profiles = joblib.load(state_path_E)

    print(f"Loading d_MI matrices (F) from: {state_path_dF}...")
    if not os.path.exists(state_path_dF):
        raise FileNotFoundError(f"State file from Cell F (d_MI) not found: {state_path_dF}")
    all_dmi_matrices = joblib.load(state_path_dF) # [FIX]

    if not all_kq_profiles or not all_dmi_matrices:
        raise ValueError("Loaded data is empty. Please run Cells E and F.")

    logger.info("Cell G dependencies (CONFIG, all_kq_profiles, all_dmi_matrices, logger) loaded.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    logger.error(traceback.format_exc())
    sys.exit(1)

# --- 2. Helper Functions (H1) ---

def compute_kq_curvature(KQ):
    """Computes |Delta^2 KQ(i)| = |KQ(i-1) - 2*KQ(i) + KQ(i+1)|"""
    # Use np.diff twice
    KQ_curv = np.diff(KQ, n=2)
    # Pad with zeros at the boundaries (sites 0 and N-1)
    KQ_curv_padded = np.pad(KQ_curv, (1, 1), 'constant', constant_values=0)
    return np.abs(KQ_curv_padded)

def compute_local_mi_pressure(d_MI_matrix, i, N, neighbors_dist=2):
    """Computes average d_MI(i, j) for j in neighbors {i+/-1, i+/-2}"""
    neighbors = []
    for dist in range(1, neighbors_dist + 1):
        if i + dist < N:
            neighbors.append(d_MI_matrix[i, i + dist])
        if i - dist >= 0:
            neighbors.append(d_MI_matrix[i, i - dist])

    if not neighbors:
        return 0.0

    return np.mean(neighbors)

# --- 3. Main Execution: Run H1 Test ---
try:
    print(f"Starting Local Geometry Test (H1) for {len(all_kq_profiles)} configurations...")

    all_h1_results = {} # Store results

    for run_id, data_KQ_df in tqdm(all_kq_profiles.items(), desc="Running H1 Test"):

        if run_id not in all_dmi_matrices:
            logger.warning(f"  Skipping {run_id}: No d_MI data found.")
            continue

        # [FIX] data_KQ is a DataFrame. Parse N/g from run_id.
        match = re.match(r"N(\d+)_g([0-9.]+)", run_id)
        if not match:
            logger.warning(f"  Skipping {run_id}: Cannot parse N and g.")
            continue
        N = int(match.group(1))
        g_val = float(match.group(2)) # Use g_val to avoid conflict

        # [FIX] Get KQ data from the DataFrame column
        if "KQ" not in data_KQ_df.columns:
            logger.warning(f"  Skipping {run_id}: DataFrame has no 'KQ' column.")
            continue
        KQ = data_KQ_df["KQ"].values

        d_MI_matrix = all_dmi_matrices[run_id]

        logger.info(f"  Processing H1 for: {run_id}")

        # Define sites to test (exclude boundaries per spec)
        sites_to_test = range(1, N - 1)
        if len(sites_to_test) < 2:
             logger.warning(f"  Skipping {run_id}: Not enough sites to test (N={N}).")
             all_h1_results[run_id] = {
                "run_id": run_id, "N": N, "g": g_val,
                "spearman_rho": np.nan, "p_rho": np.nan,
                "kendall_tau": np.nan, "p_tau": np.nan
             }
             continue

        # 1. Compute KQ Curvature vector
        KQ_curv_abs = compute_kq_curvature(KQ)
        KQ_curv_vec = [KQ_curv_abs[i] for i in sites_to_test]

        # 2. Compute Local MI Pressure vector
        dMI_pressure_vec = [compute_local_mi_pressure(d_MI_matrix, i, N) for i in sites_to_test]

        # Normalize vectors for plotting
        KQ_curv_norm = minmax_scale(KQ_curv_vec)
        dMI_pressure_norm = minmax_scale(dMI_pressure_vec)

        # 3. Calculate Correlations (Spearman & Kendall)
        try:
            # Check for constant data which causes p-value to be nan
            if np.all(KQ_curv_vec == KQ_curv_vec[0]) or np.all(dMI_pressure_vec == dMI_pressure_vec[0]):
                logger.warning(f"  Constant data found for {run_id}. Skipping correlation.")
                rho, p_rho, tau, p_tau = np.nan, np.nan, np.nan, np.nan
            else:
                rho, p_rho = spearmanr(KQ_curv_vec, dMI_pressure_vec)
                tau, p_tau = kendalltau(KQ_curv_vec, dMI_pressure_vec)
        except ValueError as ve:
             logger.warning(f"  Spearman/Kendall failed for {run_id}: {ve}")
             rho, p_rho, tau, p_tau = np.nan, np.nan, np.nan, np.nan

        logger.info(f"    H1 Results (N={N}, g={g_val:.1f}): Spearman rho = {rho:.4f} (p={p_rho:.3f})")

        # Store results
        all_h1_results[run_id] = {
            "run_id": run_id, "N": N, "g": g_val,
            "spearman_rho": rho, "p_rho": p_rho,
            "kendall_tau": tau, "p_tau": p_tau
        }

        # --- Save artifacts ---
        run_output_dir = os.path.join(OUTPUT_ROOT, run_id)

        # Save local geometry data
        df = pd.DataFrame({
            "site": sites_to_test,
            "KQ_curv_abs": KQ_curv_vec,
            "dMI_pressure": dMI_pressure_vec,
            "KQ_curv_norm": KQ_curv_norm,
            "dMI_pressure_norm": dMI_pressure_norm
        })
        df.to_csv(os.path.join(run_output_dir, "local_geometry.csv"), index=False)

        # Save scatter plot
        try:
            # [FIX] Rename plot variable to g_plot
            g_plot = sns.jointplot(x=KQ_curv_norm, y=dMI_pressure_norm, kind="reg")

            # [FIX] Use g_val (the float) not data_KQ_df['g']
            g_plot.fig.suptitle(rf"H1: Local Curvature vs MI Pressure (N={N}, g={g_val:.1f})")
            g_plot.ax_joint.set_xlabel(r"Normalized $|\Delta^2 KQ(i)|$")
            g_plot.ax_joint.set_ylabel(r"Normalized Local $d_{MI}$ Pressure")

            # Add stats text
            stats_text = (
                rf"Spearman $\rho = {rho:.3f}$ (p={p_rho:.3g})\n"
                rf"Kendall $\tau = {tau:.3f}$ (p={p_tau:.3g})"
            )
            # [FIX] Use g_plot.ax_joint
            plt.text(0.05, 0.95, stats_text, transform=g_plot.ax_joint.transAxes,
                     fontsize=10, verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            plot_path = os.path.join(run_output_dir, "local_curv_vs_pressure.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            plt.close()
            logger.info(f"    Saved H1 scatter plot for {run_id}")
        except Exception as e:
            logger.error(f"    Failed to generate H1 scatter plot for {run_id}: {e}")
            logger.error(traceback.format_exc()) # More details on plot error

    # --- 4. Save & Report Results ---
    print("\n--- Local Geometry Test (H1) Summary ---")
    print(f"Successfully processed {len(all_h1_results)} runs.")

    # Save the H1 results to a pickle file on G-Drive
    state_path = CONFIG["state_file_G"]
    joblib.dump(all_h1_results, state_path)

    print(f"\nFull H1 results data saved to: {state_path}")
    print(f"Stored all H1 results in memory (variable: all_h1_results).")

    # Show example results
    results_df = pd.DataFrame.from_dict(all_h1_results, orient='index')
    print("\nH1 Results Summary Table:")
    print(results_df.to_string(float_format="%.4f"))

    logger.info(f"Saved H1 results data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell G: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell G: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell G finished in {total_time:.2f}s.")
    print(f"\n--- [Cell G] FINISHED: Local Geometry Test (H1) ({total_time:.2f}s) ---")



--- [Cell G] START: Local Geometry Test (H1) ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:33,199 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading KQ profiles (E) from: /content/drive/MyDrive/qtx_vrd_experiment/state_E_kq_profiles.pkl...
Loading d_MI matrices (F) from: /content/drive/MyDrive/qtx_vrd_experiment/state_F_dmi_matrices.pkl...
2025-11-04 21:54:33,209 - INFO - Cell G dependencies (CONFIG, all_kq_profiles, all_dmi_matrices, logger) loaded.
Starting Local Geometry Test (H1) for 9 configurations...


Running H1 Test:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:54:33,219 - INFO -   Processing H1 for: N6_g0.5
2025-11-04 21:54:33,224 - INFO -     H1 Results (N=6, g=0.5): Spearman rho = 0.8000 (p=0.200)
2025-11-04 21:54:33,716 - INFO -     Saved H1 scatter plot for N6_g0.5
2025-11-04 21:54:33,718 - INFO -   Processing H1 for: N6_g1.0
2025-11-04 21:54:33,722 - INFO -     H1 Results (N=6, g=1.0): Spearman rho = 0.8000 (p=0.200)
2025-11-04 21:54:34,239 - INFO -     Saved H1 scatter plot for N6_g1.0
2025-11-04 21:54:34,241 - INFO -   Processing H1 for: N6_g1.5
2025-11-04 21:54:34,245 - INFO -     H1 Results (N=6, g=1.5): Spearman rho = 1.0000 (p=0.000)
2025-11-04 21:54:34,762 - INFO -     Saved H1 scatter plot for N6_g1.5
2025-11-04 21:54:34,764 - INFO -   Processing H1 for: N8_g0.5
2025-11-04 21:54:34,768 - INFO -     H1 Results (N=8, g=0.5): Spearman rho = 0.4286 (p=0.397)
2025-11-04 21:54:35,280 - INFO -     Saved H1 scatter plot for N8_g0.5
2025-11-04 21:54:35,281 - INFO -   Processing H1 for: N8_g1.0
2025-11-04 21:54:35,286 - INFO

In [24]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell H: Global Geometry Test (H2)

print("--- [Cell H] START: Global Geometry Test (H2) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, kendalltau, bootstrap
import itertools
import re # Import regex for parsing run_id

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # Load KQ profiles (E) and d_MI matrices (F)
    state_path_E = CONFIG["state_file_E"]
    state_path_dF = CONFIG["state_file_dF"] # [FIX] Load d_MI

    print(f"Loading KQ profiles (E) from: {state_path_E}...")
    if not os.path.exists(state_path_E):
        raise FileNotFoundError(f"State file from Cell E not found: {state_path_E}")
    all_kq_profiles = joblib.load(state_path_E)

    print(f"Loading d_MI matrices (F) from: {state_path_dF}...")
    if not os.path.exists(state_path_dF):
        raise FileNotFoundError(f"State file from Cell F (d_MI) not found: {state_path_dF}")
    all_dmi_matrices = joblib.load(state_path_dF) # [FIX]

    if not all_kq_profiles or not all_dmi_matrices:
        raise ValueError("Loaded data is empty. Please run Cells E and F.")

    logger.info("Cell H dependencies (CONFIG, all_kq_profiles, all_dmi_matrices, logger) loaded.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    sys.exit(1)

# --- 2. Helper Functions (H2) ---

def compute_kq_path_distance(KQ, epsilon):
    """Computes d_KQ(i,j) for all i<j"""
    N = len(KQ)

    # 1. Compute weights w(p) = eps + |grad(KQ)|
    # Use forward finite difference: |KQ(p+1) - KQ(p)|
    grad_KQ = np.abs(np.diff(KQ))
    # Pad at the end (grad[N-1] = 0)
    grad_KQ_padded = np.pad(grad_KQ, (0, 1), 'constant', constant_values=0)
    weights = epsilon + grad_KQ_padded

    # 2. Compute cumulative path distance matrix
    d_KQ_matrix = np.zeros((N, N))

    for i in range(N):
        for j in range(i + 1, N):
            # d_KQ(i,j) = sum_{p=i}^{j-1} w(p)
            path_sum = np.sum(weights[i:j])
            d_KQ_matrix[i, j] = path_sum
            d_KQ_matrix[j, i] = path_sum # Symmetric

    return d_KQ_matrix

def bootstrap_spearman(vec1, vec2, n_resamples=200):
    """Calculates bootstrap 95% CI for Spearman's rho."""
    # Scipy bootstrap requires data in a specific shape (n_obs, n_vars)
    data = np.vstack((vec1, vec2)).T

    # Define the statistic we want (Spearman's rho)
    def spearman_stat(data_sample):
        x = data_sample[:, 0]
        y = data_sample[:, 1]
        # Check for constant array, which breaks spearmanr
        if np.all(x == x[0]) or np.all(y == y[0]):
            return np.nan
        return spearmanr(x, y).correlation

    # Create bootstrap distribution
    # We pass indices to resample, as spearmanr needs paired data
    indices = (np.arange(data.shape[0]),)
    rng = np.random.default_rng(CONFIG["RNG_SEED"])

    res = bootstrap(
        indices, # Resample indices, not data directly
        lambda idx: spearman_stat(data[idx, :]),
        n_resamples=n_resamples,
        random_state=rng
    )

    # Filter out NaNs from failed runs
    dist = res.bootstrap_distribution
    dist_filtered = dist[~np.isnan(dist)]

    if len(dist_filtered) < n_resamples * 0.9: # Check if >10% failed
        logger.warning("High failure rate in bootstrap, CI may be unreliable.")

    if len(dist_filtered) == 0:
        return np.nan, np.nan

    # Get 95% CI
    ci_low = np.percentile(dist_filtered, 2.5)
    ci_high = np.percentile(dist_filtered, 97.5)

    return ci_low, ci_high

# --- 3. Main Execution: Run H2 Test ---
try:
    print(f"Starting Global Geometry Test (H2) for {len(all_kq_profiles)} configurations...")

    all_h2_results = {} # Store results
    dkq_epsilon = CONFIG["eps_path"]
    n_bootstrap = CONFIG["bootstrap_ci"]

    for run_id, data_KQ in tqdm(all_kq_profiles.items(), desc="Running H2 Test"):

        if run_id not in all_dmi_matrices:
            logger.warning(f"  Skipping {run_id}: No d_MI data found.")
            continue

        # [FIX] data_KQ is a DataFrame. Parse N/g from run_id.
        match = re.match(r"N(\d+)_g([0-9.]+)", run_id)
        if not match:
            logger.warning(f"  Skipping {run_id}: Cannot parse N and g.")
            continue
        N = int(match.group(1))
        g = float(match.group(2))

        # [FIX] Get KQ data from the DataFrame column
        if "KQ" not in data_KQ.columns:
            logger.warning(f"  Skipping {run_id}: DataFrame has no 'KQ' column.")
            continue
        KQ = data_KQ["KQ"].values

        d_MI_matrix = all_dmi_matrices[run_id]

        logger.info(f"  Processing H2 for: {run_id}")

        # 1. Compute d_KQ matrix
        d_KQ_matrix = compute_kq_path_distance(KQ, dkq_epsilon)

        # 2. Get upper-triangle pairs (i < j) for both matrices
        indices = np.triu_indices(N, k=1)
        d_KQ_vec = d_KQ_matrix[indices]
        d_MI_vec = d_MI_matrix[indices]

        if len(d_KQ_vec) == 0:
            logger.warning(f"  Skipping {run_id}: No pairs found (N={N}).")
            continue

        # 3. Calculate Correlations
        rho, p_rho = spearmanr(d_KQ_vec, d_MI_vec)
        tau, p_tau = kendalltau(d_KQ_vec, d_MI_vec)

        # 4. Calculate Bootstrap CI for Spearman's rho
        logger.info(f"    Running bootstrap (B={n_bootstrap}) for CI...")
        ci_low, ci_high = bootstrap_spearman(d_KQ_vec, d_MI_vec, n_resamples=n_bootstrap)

        logger.info(f"    H2 Results (N={N}, g={g:.1f}): Spearman rho = {rho:.4f} (p={p_rho:.3g}), 95% CI [{ci_low:.3f}, {ci_high:.3f}]")

        # Store results
        all_h2_results[run_id] = {
            "run_id": run_id, "N": N, "g": g,
            "spearman_rho": rho, "p_rho": p_rho,
            "ci_low": ci_low, "ci_high": ci_high,
            "kendall_tau": tau, "p_tau": p_tau
        }

        # --- Save artifacts ---
        run_output_dir = os.path.join(OUTPUT_ROOT, run_id)

        # Save global distance data (i, j, dKQ, dMI)
        df = pd.DataFrame({
            "i": indices[0],
            "j": indices[1],
            "dKQ": d_KQ_vec,
            "dMI": d_MI_vec
        })
        df.to_csv(os.path.join(run_output_dir, "global_distances.csv"), index=False)

        # Save scatter plot (sample if too large)
        try:
            if len(d_KQ_vec) > 2000:
                # Sample 2000 points to avoid huge plots
                rng = np.random.default_rng(CONFIG["RNG_SEED"])
                sample_indices = rng.choice(len(d_KQ_vec), 2000, replace=False)
                df_sample = df.iloc[sample_indices]
                plot_title = rf"H2: Global Distance Correlation (N={N}, g={g:.1f}) (Sampled 2k points)"
            else:
                df_sample = df
                plot_title = rf"H2: Global Distance Correlation (N={N}, g={g:.1f})"

            g = sns.jointplot(data=df_sample, x="dKQ", y="dMI", kind="reg",
                              joint_kws={'line_kws': {'color': 'red'}, 'scatter_kws': {'alpha': 0.3, 's': 10}})
            g.fig.suptitle(plot_title)

            stats_text = (
                rf"Spearman $\rho = {rho:.3f}$ (p={p_rho:.3g})\n"
                rf"95% CI: [{ci_low:.3f}, {ci_high:.3f}]"
            )
            plt.text(0.05, 0.95, stats_text, transform=g.ax_joint.transAxes,
                     fontsize=10, verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            plot_path = os.path.join(run_output_dir, "dist_scatter.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            plt.close()
            logger.info(f"    Saved H2 scatter plot for {run_id}")
        except Exception as e:
            logger.error(f"    Failed to generate H2 scatter plot for {run_id}: {e}")

    # --- 4. Save & Report Results ---
    print("\n--- Global Geometry Test (H2) Summary ---")
    print(f"Successfully processed {len(all_h2_results)} runs.")

    # Save the H2 results to a pickle file on G-Drive
    state_path = CONFIG["state_file_H"]
    joblib.dump(all_h2_results, state_path)

    print(f"\nFull H2 results data saved to: {state_path}")
    print(f"Stored all H2 results in memory (variable: all_h2_results).")

    # Show example results
    results_df = pd.DataFrame.from_dict(all_h2_results, orient='index')
    print("\nH2 Results Summary Table:")
    print(results_df.to_string(float_format="%.4f"))

    logger.info(f"Saved H2 results data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell H: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell H: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell H finished in {total_time:.2f}s.")
    print(f"\n--- [Cell H] FINISHED: Global Geometry Test (H2) ({total_time:.2f}s) ---")




--- [Cell H] START: Global Geometry Test (H2) ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:42,109 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading KQ profiles (E) from: /content/drive/MyDrive/qtx_vrd_experiment/state_E_kq_profiles.pkl...
Loading d_MI matrices (F) from: /content/drive/MyDrive/qtx_vrd_experiment/state_F_dmi_matrices.pkl...
2025-11-04 21:54:42,119 - INFO - Cell H dependencies (CONFIG, all_kq_profiles, all_dmi_matrices, logger) loaded.
Starting Global Geometry Test (H2) for 9 configurations...


Running H2 Test:   0%|          | 0/9 [00:00<?, ?it/s]

2025-11-04 21:54:42,129 - INFO -   Processing H2 for: N6_g0.5
2025-11-04 21:54:42,132 - INFO -     Running bootstrap (B=200) for CI...
2025-11-04 21:54:42,263 - INFO -     H2 Results (N=6, g=0.5): Spearman rho = 0.9821 (p=8.16e-11), 95% CI [0.908, 1.000]
2025-11-04 21:54:42,855 - INFO -     Saved H2 scatter plot for N6_g0.5
2025-11-04 21:54:42,856 - INFO -   Processing H2 for: N6_g1.0
2025-11-04 21:54:42,860 - INFO -     Running bootstrap (B=200) for CI...
2025-11-04 21:54:42,977 - INFO -     H2 Results (N=6, g=1.0): Spearman rho = 0.9705 (p=2.07e-09), 95% CI [0.843, 1.000]
2025-11-04 21:54:43,496 - INFO -     Saved H2 scatter plot for N6_g1.0
2025-11-04 21:54:43,498 - INFO -   Processing H2 for: N6_g1.5
2025-11-04 21:54:43,501 - INFO -     Running bootstrap (B=200) for CI...
2025-11-04 21:54:43,616 - INFO -     H2 Results (N=6, g=1.5): Spearman rho = 0.9286 (p=5.87e-07), 95% CI [0.687, 0.989]
2025-11-04 21:54:44,259 - INFO -     Saved H2 scatter plot for N6_g1.5
2025-11-04 21:54:44,26

In [25]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell I: Critical Sensitivity (H3)

print("--- [Cell I] START: Critical Sensitivity (H3) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import friedmanchisquare

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    print(f"Loading CONFIG from file: {config_path}")
    logger.info(f"Loading CONFIG from file: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell 0 and Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell 0 and Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    # --- [FIX END] ---

    # Load H1 (G) and H2 (H) results
    state_path_G = CONFIG["state_file_G"]
    state_path_H = CONFIG["state_file_H"]

    print(f"Loading H1 results from: {state_path_G}...")
    if not os.path.exists(state_path_G):
        raise FileNotFoundError(f"State file from Cell G not found: {state_path_G}")
    all_h1_results = joblib.load(state_path_G)

    print(f"Loading H2 results from: {state_path_H}...")
    if not os.path.exists(state_path_H):
        raise FileNotFoundError(f"State file from Cell H not found: {state_path_H}")
    all_h2_results = joblib.load(state_path_H)

    if not all_h1_results or not all_h2_results:
        raise ValueError("Loaded data is empty. Please run Cells G and H.")

    logger.info("Successfully loaded dependencies for Cell I.")

except Exception as e:
    print(f"ERROR: Failed to load dependencies: {e}")
    logger.error(f"ERROR: Failed to load dependencies: {e}")
    logger.error(traceback.format_exc())
    sys.exit(1)


# --- 2. Main Execution: Analyze H3 ---
try:
    print("Starting Critical Sensitivity analysis (H3)...")
    logger.info("Starting Critical Sensitivity analysis (H3)...")

    # Convert results to DataFrames
    # [FIX] Drop duplicate 'run_id' column before reset_index
    h1_df = pd.DataFrame.from_dict(all_h1_results, orient='index')
    if 'run_id' in h1_df.columns:
        h1_df = h1_df.drop(columns=['run_id'])
    h1_df = h1_df.reset_index().rename(columns={"index": "run_id"})

    h2_df = pd.DataFrame.from_dict(all_h2_results, orient='index')
    if 'run_id' in h2_df.columns:
        h2_df = h2_df.drop(columns=['run_id'])
    h2_df = h2_df.reset_index().rename(columns={"index": "run_id"})

    # Rename columns for merging (H1)
    h1_df = h1_df.rename(columns={
        "spearman_rho": "spearman_rho_H1", "p_rho": "p_rho_H1",
        "kendall_tau": "kendall_tau_H1", "p_tau": "p_tau_H1"
    })

    # Rename columns for merging (H2)
    h2_df = h2_df.rename(columns={
        "spearman_rho": "spearman_rho_H2", "p_rho": "p_rho_H2",
        "kendall_tau": "kendall_tau_H2", "p_tau": "p_tau_H2"
    })

    # Merge H1 and H2 results
    results_df = pd.merge(h1_df, h2_df, on=["run_id", "N", "g"], how="outer")

    print("\n--- Combined H1 & H2 Results ---")
    print(results_df.to_string(float_format="%.4f"))
    logger.info("\n--- Combined H1 & H2 Results ---\n" + results_df.to_string(float_format="%.4f"))

    # --- H3 Statistical Test (Friedman test) ---
    print("\n--- H3 Statistical Tests (Friedman) ---")
    logger.info("--- H3 Statistical Tests (Friedman) ---")

    # We test if the rho values are different across g for a fixed N
    for N_val in CONFIG["N_exact"]:
        df_n = results_df[results_df["N"] == N_val]

        # Test for H1 (Local)
        try:
            stat_h1, p_h1 = friedmanchisquare(
                df_n[df_n["g"] == 0.5]["spearman_rho_H1"],
                df_n[df_n["g"] == 1.0]["spearman_rho_H1"],
                df_n[df_n["g"] == 1.5]["spearman_rho_H1"]
            )
            print(f"H3 Test (H1 rho, N={N_val}): Friedman stat={stat_h1:.3f}, p-value={p_h1:.4f}")
            logger.info(f"H3 Test (H1 rho, N={N_val}): Friedman stat={stat_h1:.3f}, p-value={p_h1:.4f}")
        except Exception as e:
            print(f"H3 Test (H1 rho, N={N_val}): FAILED ({e})")
            logger.warning(f"H3 Test (H1 rho, N={N_val}): FAILED ({e})")

        # Test for H2 (Global)
        try:
            stat_h2, p_h2 = friedmanchisquare(
                df_n[df_n["g"] == 0.5]["spearman_rho_H2"],
                df_n[df_n["g"] == 1.0]["spearman_rho_H2"],
                df_n[df_n["g"] == 1.5]["spearman_rho_H2"]
            )
            print(f"H3 Test (H2 rho, N={N_val}): Friedman stat={stat_h2:.3f}, p-value={p_h2:.4f}")
            logger.info(f"H3 Test (H2 rho, N={N_val}): Friedman stat={stat_h2:.3f}, p-value={p_h2:.4f}")
        except Exception as e:
            print(f"H3 Test (H2 rho, N={N_val}): FAILED ({e})")
            logger.warning(f"H3 Test (H2 rho, N={N_val}): FAILED ({e})")

    # --- H3 Trend Plot ---
    print("\n--- Generating H3 Trend Plots ---")
    logger.info("--- Generating H3 Trend Plots ---")

    # Melt dataframe for easier plotting with seaborn
    df_melt = results_df.melt(
        id_vars=["run_id", "N", "g"],
        value_vars=["spearman_rho_H1", "spearman_rho_H2"],
        var_name="Hypothesis",
        value_name="Spearman_rho"
    )
    df_melt["Hypothesis"] = df_melt["Hypothesis"].map({
        "spearman_rho_H1": "H1 (Local)",
        "spearman_rho_H2": "H2 (Global)"
    })

    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

    # H1 Plot
    sns.lineplot(data=results_df, x="g", y="spearman_rho_H1", hue="N", style="N", markers=True, ax=ax1, palette="deep")
    # [FIX] Use raw string r"..."
    ax1.set_title(r"H3: H1 (Local $|\Delta^2 KQ|$) $\rho$ vs. g")
    ax1.set_ylabel(r"Spearman $\rho$")
    ax1.set_xlabel("g (coupling)")
    ax1.axvline(1.0, color='r', linestyle='--', label="Critical Point (g=1)")
    ax1.legend(title="N")
    ax1.grid(True, linestyle=':')

    # H2 Plot
    sns.lineplot(data=results_df, x="g", y="spearman_rho_H2", hue="N", style="N", markers=True, ax=ax2, palette="deep")
    # [FIX] Use raw string r"..."
    ax2.set_title(r"H3: H2 (Global $d_{KQ}$) $\rho$ vs. g")
    ax2.set_ylabel(r"Spearman $\rho$")
    ax2.set_xlabel("g (coupling)")
    ax2.axvline(1.0, color='r', linestyle='--', label="Critical Point (g=1)")
    ax2.legend(title="N")
    ax2.grid(True, linestyle=':')

    # [FIX] Use raw string r"..."
    fig.suptitle(r"H3: Critical Sensitivity Trend (Spearman $\rho$)", fontsize=16, y=1.03)
    plt.tight_layout()

    plot_path = os.path.join(OUTPUT_ROOT, "h3_critical_sensitivity.png")
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close()

    print(f"Saved H3 trend plot to: {plot_path}")
    logger.info(f"Saved H3 trend plot to: {plot_path}")

    # --- 4. Save & Report Results ---
    print("\n--- H3 Analysis Summary ---")

    # Save the H3 results to a pickle file on G-Drive
    state_path = CONFIG["state_file_I"]
    joblib.dump(results_df, state_path)

    print(f"\nFull H3 (combined) results data saved to: {state_path}")
    print(f"Stored H3 results in memory (variable: results_df).")
    logger.info(f"Saved H3 results data to {state_path}")

except Exception as e:
    logger.error(f"An error occurred in Cell I: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell I: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell I finished in {total_time:.2f}s.")
    print(f"\n--- [Cell I] FINISHED: Critical Sensitivity (H3) ({total_time:.2f}s) ---")



--- [Cell I] START: Critical Sensitivity (H3) ---
Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:54:51,840 - INFO - Loading CONFIG from file: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading H1 results from: /content/drive/MyDrive/qtx_vrd_experiment/state_G_h1_results.pkl...
Loading H2 results from: /content/drive/MyDrive/qtx_vrd_experiment/state_H_h2_results.pkl...
2025-11-04 21:54:51,847 - INFO - Successfully loaded dependencies for Cell I.
Starting Critical Sensitivity analysis (H3)...
2025-11-04 21:54:51,849 - INFO - Starting Critical Sensitivity analysis (H3)...

--- Combined H1 & H2 Results ---
     run_id   N      g  spearman_rho_H1  p_rho_H1  kendall_tau_H1  p_tau_H1  spearman_rho_H2  p_rho_H2  ci_low  ci_high  kendall_tau_H2  p_tau_H2
0  N10_g0.5  10 0.5000           0.7381    0.0366          0.5000    0.1087           0.9951    0.0000  0.9836   0.9971          0.9469    0.0000
1  N10_g1.0  10 1.0000           0.7619 

In [26]:
# Author: Artem Brezgin, Spanda Foundation (C) 2025
# Cell J: Ablation Robustness Tests (H4)

print("--- [Cell J] START: Ablation Robustness Tests (H4) ---")

import logging
import os
import sys
import numpy as np
import pandas as pd
import joblib
from tqdm.auto import tqdm
import time
import traceback
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import minmax_scale
import re # Import regex for parsing run_id
import itertools

# --- 1. Load Config & Dependencies ---
try:
    start_time = time.time()

    # Setup logger
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger(__name__)

    # --- [FIX] Load CONFIG from G-Drive file ---
    print("Loading CONFIG from file...")
    DRIVE_ROOT = "/content/drive/MyDrive/qtx_vrd_experiment"
    config_path = os.path.join(DRIVE_ROOT, "config.json")

    # [FIX] Add logging to show *which* file is being checked
    print(f"  Attempting to load config from absolute path: {config_path}")
    logger.info(f"  Attempting to load config from absolute path: {config_path}")

    if not os.path.exists(config_path):
        print("ERROR: CONFIG file not found. Please run Cell A first.")
        raise FileNotFoundError("CONFIG file not found. Please run Cell A first.")

    import json
    with open(config_path, 'r') as f:
        CONFIG = json.load(f)

    OUTPUT_ROOT = CONFIG["output_root"]
    EPS_LOG = CONFIG.get("eps_log", 1e-8)
    EPS_PATH = CONFIG.get("eps_path", 1e-4)
    # --- [FIX END] ---

    # --- Load all required states ---
    print("Loading all required states (B, C, D, E, F, G, H)...")
    logger.info("Loading all required states (B, C, D, E, F, G, H)...")

    ground_states_exact = joblib.load(CONFIG["state_file_B"])
    all_rdm_data = joblib.load(CONFIG["state_file_C"])
    all_structure_profiles = joblib.load(CONFIG["state_file_D"])
    all_kq_profiles = joblib.load(CONFIG["state_file_E"])
    all_mi_matrices = joblib.load(CONFIG["state_file_F"])
    all_dmi_matrices = joblib.load(CONFIG["state_file_dF"]) # Corrected path
    all_h1_results = joblib.load(CONFIG["state_file_G"])
    all_h2_results = joblib.load(CONFIG["state_file_H"])

    print("Successfully loaded all dependencies.")
    logger.info("Successfully loaded all dependencies.")

except Exception as e:
    # [FIX] More specific error message
    error_msg = f"ERROR: State file not found: {e}. Please run previous cells first."
    print(error_msg)
    logger.error(error_msg)
    logger.error(traceback.format_exc())
    sys.exit(1) # Stop execution if dependencies are missing


# --- 2. Self-Contained Helper Functions (Copied for Robustness) ---
# These are copied from Cells C, D, E, G, H to make this cell runnable
# (since functions defined in other cells are not in memory)

def compute_von_neumann_entropy(rho):
    eigvals = np.linalg.eigvalsh(rho)
    eigvals = eigvals[eigvals > 1e-15] # Avoid log(0)
    return -np.sum(eigvals * np.log2(eigvals))

def get_exp_val_rho(rho_i, op):
    return np.real(np.trace(rho_i @ op))

def get_connected_correlator(psi_0, op1, op2, i, j, N):
    # This function is not fully implemented here as it's complex
    # We rely on the re-computation of C(i) below.
    pass

def compute_local_mi_pressure(d_MI_matrix, i, N, neighbors_dist=2):
    neighbors = []
    for dist in range(1, neighbors_dist + 1):
        if i + dist < N:
            neighbors.append(d_MI_matrix[i, i + dist])
        if i - dist >= 0:
            neighbors.append(d_MI_matrix[i, i - dist])
    return np.mean(neighbors) if neighbors else 0.0

def compute_kq_curvature(KQ):
    KQ_curv = np.diff(KQ, n=2)
    KQ_curv_padded = np.pad(KQ_curv, (1, 1), 'constant', constant_values=0)
    return np.abs(KQ_curv_padded)

def compute_kq_path_distance(KQ, epsilon):
    N = len(KQ)
    grad_KQ = np.abs(np.diff(KQ))
    grad_KQ_padded = np.pad(grad_KQ, (0, 1), 'constant', constant_values=0)
    weights = epsilon + grad_KQ_padded
    d_KQ_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(i + 1, N):
            path_sum = np.sum(weights[i:j])
            d_KQ_matrix[i, j] = path_sum
            d_KQ_matrix[j, i] = path_sum
    return d_KQ_matrix

# --- 3. Main Ablation Logic ---

def run_ablation_c_definition(run_id, N, g, c_type='ZZ'):
    """
    Re-computes KQ, H1, and H2 for a modified C(i) definition.
    c_type: 'ZZ', 'XX', or 'NN' (next-neighbor)
    """

    # Get pre-computed data
    structure_df = all_structure_profiles[run_id]

    # [FIX] Use the correct key 'H_norm_list'
    H_norm_list = all_rdm_data[run_id]["H_norm_list"]

    dmi_matrix = all_dmi_matrices[run_id]

    # [FIX] Ensure H_norm is a numpy array
    H_norm = np.array(H_norm_list)

    # 1. Re-calculate C(i)
    if c_type == 'ZZ':
        C_raw = structure_df["C_raw_ZZ"].values
    elif c_type == 'XX':
        C_raw = structure_df["C_raw_XX"].values
    # TODO: Add 'NN' logic if needed
    else:
        # Default (canonical)
        C_raw = np.mean([
            structure_df["C_raw_ZZ"].values,
            structure_df["C_raw_XX"].values
        ], axis=0)

    C_norm = minmax_scale(C_raw)

    # 2. Re-calculate KQ
    KQ = C_norm * (1 - H_norm)

    # 3. Re-calculate H1 (Local)
    sites_to_test = range(1, N - 1)
    if len(sites_to_test) < 2:
        return np.nan, np.nan

    KQ_curv_vec = [compute_kq_curvature(KQ)[i] for i in sites_to_test]
    dMI_pressure_vec = [compute_local_mi_pressure(dmi_matrix, i, N) for i in sites_to_test]

    rho_h1, _ = spearmanr(KQ_curv_vec, dMI_pressure_vec)

    # 4. Re-calculate H2 (Global)
    d_KQ_matrix = compute_kq_path_distance(KQ, EPS_PATH)
    indices = np.triu_indices(N, k=1)
    d_KQ_vec = d_KQ_matrix[indices]
    d_MI_vec = dmi_matrix[indices]

    rho_h2, _ = spearmanr(d_KQ_vec, d_MI_vec)

    return rho_h1, rho_h2

# (Future: Add run_ablation_mi_variant)

# --- 4. Main Execution: Run Ablations (H4) ---
try:
    print(f"Starting H4 Ablation tests for {len(all_h1_results)} configurations...")
    logger.info(f"--- [Cell J] Starting Ablation Robustness Tests (H4) ---")

    ablation_results = []

    for run_id in tqdm(all_h1_results.keys(), desc="Running H4 Ablations"):

        # [FIX] Parse N/g from run_id
        match = re.match(r"N(\d+)_g([0-9.]+)", run_id)
        if not match:
            logger.warning(f"Could not parse run_id: {run_id}. Skipping.")
            continue
        N = int(match.group(1))
        g_val = float(match.group(2))

        # Get baseline correlations
        baseline_h1 = all_h1_results[run_id]['spearman_rho']
        baseline_h2 = all_h2_results[run_id]['spearman_rho']

        # Run Ablation: C(i) = ZZ only
        rho_h1_zz, rho_h2_zz = run_ablation_c_definition(run_id, N, g_val, c_type='ZZ')

        # Run Ablation: C(i) = XX only
        rho_h1_xx, rho_h2_xx = run_ablation_c_definition(run_id, N, g_val, c_type='XX')

        # (Future: Renyi-2 Ablation)

        ablation_results.append({
            "run_id": run_id,
            "N": N,
            "g": g_val,
            "H1_baseline": baseline_h1,
            "H1_C_ZZ": rho_h1_zz,
            "H1_C_XX": rho_h1_xx,
            "H2_baseline": baseline_h2,
            "H2_C_ZZ": rho_h2_zz,
            "H2_C_XX": rho_h2_xx,
        })

    # --- 5. Save & Report Results ---
    print("\n--- Ablation Test (H4) Summary ---")

    results_df_h4 = pd.DataFrame(ablation_results)
    print(results_df_h4.to_string(float_format="%.4f"))

    # Save the H4 results to G-Drive
    state_path = CONFIG["state_file_J"]
    joblib.dump(results_df_h4, state_path)

    print(f"\nFull H4 (Ablation) results data saved to: {state_path}")
    print(f"Stored H4 results in memory (variable: results_df_h4).")
    logger.info(f"Saved H4 results data to {state_path}")

    # --- Plot H4 Robustness ---
    print("Generating H4 Robustness Plots...")
    try:
        df_melt = results_df_h4.melt(
            id_vars=["run_id", "N", "g"],
            value_vars=["H2_baseline", "H2_C_ZZ", "H2_C_XX"],
            var_name="Ablation_Type",
            value_name="H2_Spearman_rho"
        )

        g_plot = sns.catplot(
            data=df_melt, x="g", y="H2_Spearman_rho",
            hue="Ablation_Type", col="N",
            kind="point", palette="muted",
            height=5, aspect=0.8
        )

        g_plot.fig.suptitle("H4: Ablation Robustness (H2 Global Correlation)", y=1.05)
        g_plot.set_axis_labels("g (coupling)", r"Spearman $\rho$")

        plot_path = os.path.join(OUTPUT_ROOT, "h4_ablation_robustness.png")
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        plt.close()
        print(f"Saved H4 robustness plot to: {plot_path}")
        logger.info(f"Saved H4 robustness plot to: {plot_path}")

    except Exception as e:
        logger.error(f"Failed to generate H4 robustness plots: {e}")

except Exception as e:
    logger.error(f"An error occurred in Cell J: {e}")
    logger.error(traceback.format_exc())
    print(f"ERROR in Cell J: {e}")

finally:
    total_time = time.time() - start_time
    logger.info(f"Cell J finished in {total_time:.2f}s.")
    print(f"\n--- [Cell J] FINISHED: Ablation Tests ({total_time:.2f}s) ---")



--- [Cell J] START: Ablation Robustness Tests (H4) ---
Loading CONFIG from file...
  Attempting to load config from absolute path: /content/drive/MyDrive/qtx_vrd_experiment/config.json
2025-11-04 21:55:03,639 - INFO -   Attempting to load config from absolute path: /content/drive/MyDrive/qtx_vrd_experiment/config.json
Loading all required states (B, C, D, E, F, G, H)...
2025-11-04 21:55:03,642 - INFO - Loading all required states (B, C, D, E, F, G, H)...
Successfully loaded all dependencies.
2025-11-04 21:55:03,666 - INFO - Successfully loaded all dependencies.
Starting H4 Ablation tests for 9 configurations...
2025-11-04 21:55:03,669 - INFO - --- [Cell J] Starting Ablation Robustness Tests (H4) ---


Running H4 Ablations:   0%|          | 0/9 [00:00<?, ?it/s]


--- Ablation Test (H4) Summary ---
     run_id   N      g  H1_baseline  H1_C_ZZ  H1_C_XX  H2_baseline  H2_C_ZZ  H2_C_XX
0   N6_g0.5   6 0.5000       0.8000   0.8000  -0.8000       0.9821   0.9964   0.9857
1   N6_g1.0   6 1.0000       0.8000   0.6000   0.8000       0.9705   0.9705   0.8525
2   N6_g1.5   6 1.5000       1.0000   1.0000   0.8000       0.9286   0.9214   0.7500
3   N8_g0.5   8 0.5000       0.4286   0.4857   0.9429       0.9933   0.9982   0.9966
4   N8_g1.0   8 1.0000       0.9856   0.9856   0.4638       0.9145   0.9145   0.8466
5   N8_g1.5   8 1.5000       0.4286   0.4286   0.4286       0.8068   0.8057   0.7997
6  N10_g0.5  10 0.5000       0.7381   0.7857   0.9762       0.9951   0.9943   0.9975
7  N10_g1.0  10 1.0000       0.7619   0.7381   0.3810       0.8352   0.8354   0.8021
8  N10_g1.5  10 1.5000       0.0238   0.0476   0.0238       0.7436   0.7436   0.7411

Full H4 (Ablation) results data saved to: /content/drive/MyDrive/qtx_vrd_experiment/state_J_h4_results.pkl
Stored